In [ ]:
import pandas as pd
import io
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
from pathlib import Path
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

pd.options.display.max_columns = None


In [ ]:
csv_paths = [
    "baseline_out.csv",
    "final_out.csv",
    "transfer_out.csv"
]

# csv_paths = [Path('talos/final', path) for path in csv_paths]

df = pd.DataFrame([])
for path in csv_paths:
    with open(path, 'r') as fi:
        data = fi.read().replace('(', '"').replace(')', '"')
        df = pd.concat([df, pd.read_csv(io.StringIO(data))])

df['input_shape'] = df['input_shape'].str.replace(',', '')
df

# exclude = ['round_epochs', 'loss', 'acc', 'f1score', 'val_loss', 'val_acc', 'val_f1score']
# include = list(set(df.columns) - set(exclude))
# df = df.drop_duplicates(subset=include)

## Create difference between train and validation for loss and accuracy 
df.insert(7, 'train_val_loss_diff', df['val_loss'].astype(float) - df['loss'])
df.insert(8, 'train_val_acc_diff', df['val_acc'].astype(float) - df['acc'])
df.insert(9, 'train_val_f1score_diff', df['val_f1score'].astype(float) - df['f1score'])

## Remove all columns with only single value
for col in df.columns:
    if len(df[col].unique()) == 1:
        df.drop(col, inplace=True, axis=1)

df = df.sort_values('val_f1score', ascending=False)
df

In [ ]:
def print_boxplots(df,param):
    fig, axes = plt.subplots(3, 3, figsize=(18, 10))
    fig.suptitle(f'Stats for: {param}')

    sns.boxplot(ax=axes[0, 0], data=df, x=param, y='acc')
    sns.boxplot(ax=axes[1, 0], data=df, x=param, y='val_acc')
    sns.boxplot(ax=axes[2, 0], data=df, x=param, y='train_val_acc_diff')
    sns.boxplot(ax=axes[0, 1], data=df, x=param, y='loss')
    sns.boxplot(ax=axes[1, 1], data=df, x=param, y='val_loss')
    sns.boxplot(ax=axes[2, 1], data=df, x=param, y='train_val_loss_diff')
    sns.boxplot(ax=axes[0, 2], data=df, x=param, y='f1score')
    sns.boxplot(ax=axes[1, 2], data=df, x=param, y='val_f1score')
    sns.boxplot(ax=axes[2, 2], data=df, x=param, y='train_val_f1score_diff')
    return 

exclude = ['round_epochs', 'loss', 'acc', 'f1score', 'val_loss', 'val_acc', 'val_f1score', 'train_val_loss_diff', 'train_val_acc_diff', "train_val_f1score_diff"]
boxplot_columns = [column for column in df.columns if column not in exclude ] 
for var in boxplot_columns:
    print_boxplots(df,var)

In [ ]:
df_num = df.drop(columns=['input_shape'])

left = ['round_epochs', 'loss', 'acc', 'f1score', 'val_loss', 'val_acc', 'val_f1score', 'train_val_loss_diff', 'train_val_acc_diff', "train_val_f1score_diff"]
bottom = list(set(df_num.columns) - set(left))

fig, ax = plt.subplots(figsize=(8,8))
corrMatrix = df_num.corr()
# print(corrMatrix)
corrMatrix = corrMatrix.drop(labels=left, axis=1)
corrMatrix = corrMatrix.drop(labels=bottom, axis=0)
sn.heatmap(corrMatrix, annot=True)
plt.show()